In [ ]:
import tensorflow as tf
import keras

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
print(tf.__version__)
print(keras.__version__)
print (np.__version__)

In [ ]:
# Hyperparams if GPU is available
if tf.test.is_gpu_available():
    BATCH_SIZE = 512  # Number of examples used in each iteration
    EPOCHS = 150  # Number of passes through entire dataset
    MAX_LEN = 2000  # Max length of review (in words)
    EMBEDDING = 40  # Dimension of word embedding vector

    
# Hyperparams for CPU training
else:
    BATCH_SIZE = 32
    EPOCHS = 250
    MAX_LEN = 500
    EMBEDDING = 300

In [ ]:
data = pd.read_excel(r"C:\Users\sanjana.suman\Desktop\GoverningLawAndJurisdiction\improved\ibotagged12.xlsx")
data = data.fillna(method="ffill")

print("Number of sentences: ", len(data.groupby(['Sentence #'])))

words = list(set(data["Word"].values))
n_words = len(words)
print("Number of words in the dataset: ", n_words)

tags = list(set(data["Tag"].values))
print("Tags:", tags)
n_tags = len(tags)
print("Number of Labels: ", n_tags)

print("What the dataset looks like:")
# Show the first 10 rows
data.head(n=10)

In [ ]:

class SentenceGetter(object):
    """Class to Get the sentence in this format:
    [(Token_1, Part_of_Speech_1, Tag_1), ..., (Token_n, Part_of_Speech_1, Tag_1)]"""
    def __init__(self, data):
        """Args:
            data is the pandas.DataFrame which contains the above dataset"""
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        """Return one sentence"""
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None
        
getter = SentenceGetter(data)
sent = getter.get_next()
print('This is what a sentence looks like:')
print(sent)

In [ ]:
sentences = getter.sentences

# Plot sentence by lenght
plt.hist([len(s) for s in sentences], bins=50)
plt.title('Token per sentence')
plt.xlabel('Len (number of token)')
plt.ylabel('# samples')
plt.show()

In [ ]:
max_len = 500
max_len_char = 20

In [ ]:
word2idx = {w: i + 2 for i, w in enumerate(words)}
word2idx["UNK"] = 1
word2idx["PAD"] = 0
idx2word = {i: w for w, i in word2idx.items()}
tag2idx = {t: i + 1 for i, t in enumerate(tags)}
tag2idx["PAD"] = 0
idx2tag = {i: w for w, i in tag2idx.items()}

In [ ]:
print("The word laws is identified by the index: {}".format(word2idx["law"]))
print("The labelsI-Juris (which defines Geopraphical Enitities) is identified by the index: {}".format(tag2idx["I-Juris"]))


In [ ]:
from keras.preprocessing.sequence import pad_sequences
# Convert each sentence from list of Token to list of word_index
X = [[word2idx[w[0]] for w in s] for s in sentences]
# Padding each sentence to have the same lenght
X = pad_sequences(maxlen=MAX_LEN, sequences=X,value=word2idx["PAD"], padding="post", truncating='post')

In [ ]:
chars = set([w_i for w in words for w_i in str(w)])
n_chars = len(chars)
print(n_chars)

In [ ]:
char2idx = {c: i + 2 for i, c in enumerate(chars)}
char2idx["UNK"] = 1
char2idx["PAD"] = 0

In [ ]:
X_char = []
for sentence in sentences:
    sent_seq = []
    for i in range(max_len):
        word_seq = []
        for j in range(max_len_char):
            try:
                word_seq.append(char2idx.get(sentence[i][0][j]))
            except:
                word_seq.append(char2idx.get("PAD"))
        sent_seq.append(word_seq)
    X_char.append(np.array(sent_seq))

In [ ]:
y = [[tag2idx[w[2]] for w in s] for s in sentences]

In [ ]:
y = pad_sequences(maxlen=max_len, sequences=y, value=tag2idx["PAD"], padding='post', truncating='post')

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:

X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.1, random_state=2018)
X_char_tr, X_char_te, _, _ = train_test_split(X_char, y, test_size=0.1, random_state=2018)

In [ ]:
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Conv1D
from keras.layers import Bidirectional, concatenate, SpatialDropout1D, GlobalMaxPooling1D

In [ ]:
# input and embedding for words
word_in = Input(shape=(max_len,))
emb_word = Embedding(input_dim=n_words + 2, output_dim=20,
                     input_length=max_len, mask_zero=True)(word_in)

# input and embeddings for characters
char_in = Input(shape =(max_len, max_len_char,))
emb_char = TimeDistributed(Embedding(input_dim=n_chars + 2, output_dim=20,
                           input_length=max_len_char, mask_zero=True))(char_in)
# character LSTM to get word encodings by characters
char_enc = TimeDistributed(LSTM(units=20, return_sequences=False,
                                recurrent_dropout=0.5))(emb_char)

# main LSTM
x = concatenate([emb_word, char_enc])
x = SpatialDropout1D(0.3)(x)
main_lstm = Bidirectional(LSTM(units=50, return_sequences=True,
                               recurrent_dropout=0.6))(x)
out = TimeDistributed(Dense(n_tags + 1, activation="softmax"))(main_lstm)

model = Model([word_in, char_in], out)

In [ ]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["acc"])

In [ ]:
model.summary()

In [ ]:
print(max_len)

In [ ]:
history = model.fit([X_tr,
                     np.array(X_char_tr).reshape((len(X_char_tr), max_len, max_len_char))],
                    np.array(y_tr).reshape(len(y_tr), max_len, 1),
                    batch_size=32, epochs=250, validation_split=0.1, verbose=1)

In [ ]:
hist = pd.DataFrame(history.history)

In [ ]:
import matplotlib.pyplot as plt
plt.style.use("ggplot")
plt.figure(figsize=(12,12))
plt.plot(hist["acc"])
plt.plot(hist["val_acc"])
plt.show()

In [ ]:
y_pred = model.predict([X_te,
                        np.array(X_char_te).reshape((len(X_char_te),
                                                     max_len, max_len_char))])

In [ ]:
i = np.random.randint(0,X_te.shape[0])
p = np.argmax(y_pred[i], axis=-1)

print("{:15}||{:5}||{}".format("Word", "True", "Pred"))
print(30 * "=")
for w, t, pred in zip(X_te[i], y_te[i], p):
    if w != 0:
        print("{:15}: {:5} {}".format(idx2word[w], idx2tag[t], idx2tag[pred]))

In [ ]:
import pickle
from keras.models import load_model
from keras_contrib.utils import save_load_utils

# Saving Vocab
with open(r'C:\Users\sanjana.suman\Desktop\GoverningLawAndJurisdiction\improved\bilm+lm\20words\word_to_index12.pickle', 'wb') as handle:
    pickle.dump(word2idx, handle, protocol=pickle.HIGHEST_PROTOCOL)
# Saving Vocab
with open(r'C:\Users\sanjana.suman\Desktop\GoverningLawAndJurisdiction\improved\bilm+lm\20words\tag_to_index12.pickle', 'wb') as handle:
    pickle.dump(tag2idx, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(r'C:\Users\sanjana.suman\Desktop\GoverningLawAndJurisdiction\improved\bilm+lm\20words\char2idx12.pickle', 'wb') as handle:
    pickle.dump(char2idx, handle, protocol=pickle.HIGHEST_PROTOCOL)    
# Saving Model Weight
model.save_weights(r'C:\Users\sanjana.suman\Desktop\GoverningLawAndJurisdiction\improved\bilm+lm\20words\lstm_crf_weights12.h5')
model.save(r'C:\Users\sanjana.suman\Desktop\GoverningLawAndJurisdiction\improved\bilm+lm\20words\lstm_crf_weights12.h5',)

In [ ]:
from keras_contrib.layers import CRF
from keras.models import load_model
import h5py
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_viterbi_accuracy
import pickle


def load_keras_model(path):
    model = load_model(path, custom_objects={'CRF':CRF, 
                                                  'crf_loss':crf_loss, 
                                                  'crf_viterbi_accuracy':crf_viterbi_accuracy}, compile=False)
    return model
    
path=r'C:\Users\sanjana.suman\Desktop\GoverningLawAndJurisdiction\improved\bilm+lm\20words\lstm_crf_weights12.h5' 
final_model=load_keras_model(path)
print('Model loaded. Start serving...')


print(final_model.summary())

with open(r'C:\Users\sanjana.suman\Desktop\GoverningLawAndJurisdiction\improved\bilm+lm\20words\word_to_index12.pickle', "rb") as handle:
    word2idx= pickle.load(handle)

with open(r'C:\Users\sanjana.suman\Desktop\GoverningLawAndJurisdiction\improved\bilm+lm\20words\tag_to_index12.pickle', 'rb') as handle:
    tag2idx= pickle.load(handle)
    
with open(r'C:\Users\sanjana.suman\Desktop\GoverningLawAndJurisdiction\improved\bilm+lm\20words\char2idx12.pickle', 'rb') as handle:
    char2idx= pickle.load(handle)

In [2]:
from ipywidgets import interact_manual
from ipywidgets import widgets
from nltk import word_tokenize, sent_tokenize
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import re
import string
MAX_LEN = 500

idx2tag = {i: w for w, i in tag2idx.items()}



def tokenize(sentence):
    print(word_tokenize(sentence))
    return word_tokenize(sentence)

def convert_word_to_char(predict_word):
    predict_char = []
    sent_seq = []
    for i in range(max_len):
        word_seq = []
        for j in range(max_len_char):    
            try:
                if(predict_word[i][j] in char2idx):
                    word_seq.append(char2idx.get(predict_word[i][j]))
                else:
                    print(predict_word[i][j])
                    word_seq.append(char2idx.get("UNK"))
            except:
                word_seq.append(char2idx.get("PAD"))
        sent_seq.append(word_seq)
    predict_char.append(np.array(sent_seq))
    
    return predict_char
    
def get_prediction(sentence):
    test_sentence = word_tokenize(sentence) # Tokenization
    print(test_sentence)
    # Preprocessing
    x_test_sent = pad_sequences(sequences=[[word2idx.get(w, 0) for w in test_sentence]],
                            padding="post", value=word2idx["PAD"], maxlen=MAX_LEN)
    # Evaluation
    #p = final_model.predict(np.array([x_test_sent[0]]))
    
    X_char_te=convert_word_to_char(test_sentence)
    p = final_model.predict([x_test_sent,
                        np.array(X_char_te).reshape((len(X_char_te),
                                                     max_len, max_len_char))])
    p = np.argmax(p, axis=-1)

    # Visualization
    #print([x_test_sent[0]])
    #print(p[0])
    #print("{:15}||{}".format("Word", "Prediction"))
    #print(30 * "=")
    gov_law_string=[]
    jurisdiction_string=[]
    temp=''
    gov_key1='B-gov'
    gov_key2='I-gov'
    juris_key1='B-Juris'
    juris_key2='I-Juris'
    temp1=''
    out=[]
    out1=[]
    gov_law_string_final=[]
    jurisdiction_string_final=[]
    for w, pred in zip(test_sentence, p[0]):
        print("{:15}: {:5}".format(w, idx2tag[pred]))
        if idx2tag[pred]== gov_key1 or idx2tag[pred]== gov_key2:
            if not temp:
                temp=temp+w
            else:
                temp=temp+' '+w
        elif idx2tag[pred]==juris_key1 or idx2tag[pred]==juris_key2:
            if not temp1:
                temp1=temp1+w
            else:
                temp1=temp1+' '+w
        else:
            if temp:
                out.append(temp)
            temp=''
            if temp1:
                out1.append(temp1)
            temp1=''
    if temp:
        out.append(temp)
    if temp1:
        out1.append(temp1)
    #print('Governing Law:',gov_law_string)
    gov_law_string=out
    jurisdiction_string=out1
    print(gov_law_string)
    print(jurisdiction_string)
    if len(gov_law_string)==0:
        model_output_governing_law.append('None')
    else:
        for ele in gov_law_string:
            if len(ele.split(" "))!=1:
                if 'law' in ele or 'STATE' in ele or 'state' in ele or'State' in ele or'LAW' in ele or 'Law' in ele or 'lsws' in ele:
                    gov_law_string_final.append(ele)
                else:
                    pass
                
            else:
                pass
        if len(gov_law_string_final)!=0:
                model_output_governing_law.append(gov_law_string_final)
                print('gov:',gov_law_string_final)
        else:
            model_output_governing_law.append('None')
            
    if len(jurisdiction_string)==0:
    #if " " not in jurisdiction_string:
        model_output_jurisdiction.append('None')
    else:
        for ele1 in jurisdiction_string:
            if len(ele1.split(" "))!=1:
                if 'court' in ele1 or 'cou' in ele1 or 'COURT' in ele1 or'juri' in ele1 or'Cour' in ele1 or 'state' in ele1 or 'State' in ele1 or 'Cou' in ele1 or 'services is organized.' in ele1:
                    jurisdiction_string_final.append(ele1)
                else:
                    pass
            else:
                pass
        if len(jurisdiction_string_final)!=0:
            
            model_output_jurisdiction.append(jurisdiction_string_final)
            print('Juris:',jurisdiction_string_final)
        else:
            model_output_jurisdiction.append("None")
       
    return model_output_governing_law,model_output_jurisdiction
#sentence='any dispute under this Agreement will only be brought in the state and federal courts located in the State of New York.'
interact_manual(get_prediction, sentence=widgets.Textarea(placeholder='State of New York, without giving effect to principles of conflicts of law'))

interactive(children=(Textarea(value='', description='sentence', placeholder='State of New York, without givin…

<function __main__.get_prediction(sentence)>

In [3]:
import pandas as pd


df= pd.read_excel(r"C:\Users\sanjana.suman\Desktop\mathan_gov_law\Gov_law_states_list.xlsx")
Paragraphs=df['paragraphs'].tolist()

In [4]:
model_output_governing_law=[]
model_output_jurisdiction=[]
max_len=500
max_len_char=20
for i in Paragraphs:
    get_prediction(i)
    
    
df['Governing_Law']=model_output_governing_law
df['Jurisdiction']=model_output_jurisdiction

df.to_excel(r"C:\Users\sanjana.suman\Desktop\test1.xlsx",index=False) 
print('done')

['SECTION', '6', 'GENERAŁ', 'PROVISIONS', '.', 'fi.1', 'Protection', 'of', 'Client', 'Files', '.', 'ADP', 'will', 'take', 'reasonable', 'precautions', 'to', 'prevent', 'the', 'loss', 'of', 'or', 'alteration', 'to', 'Client', "'s", 'data', 'files', 'In', 'ADP', '’', 's', 'possassion', '.', 'Client', 'wit', ']', ',', 'to', 'the', 'extent', 'it', 'deems', 'necessary', ',', 'kaep', 'copies', 'of', 'all', 'soume', 'dauments', 'of', 'me', 'infomat', ')', 'on', 'delivered', 'to', 'ADP', 'or', 'inputted', 'by', 'Client', 'or', 'on', 'behalf', 'of', 'Client', 'into', 'the', 'ADP', 'system', 'and', 'will', 'maintain', 'a', 'procedure', 'external', 'to', 'the', 'ADP', 'system', 'for', 'the', 'reconstruction', 'of', 'lost', 'or', 'altered', 'data', 'files', '.', 'If', 'requested', 'by', 'Client', ',', 'ADP', 'will', 'deliver', 'to', 'Client', '(', 'in', 'a', 'gECTION', '4', '\x0eDISCLAFMER', 'OF', 'WARRANTTES', '.', '\x0estandard', 'ADP', 'formal', 'and', 'at', 'ADP', '’', 'a', 'then', 'standard',

SECTION        : O    
6              : O    
GENERAŁ        : O    
PROVISIONS     : O    
.              : O    
fi.1           : O    
Protection     : O    
of             : O    
Client         : O    
Files          : O    
.              : O    
ADP            : O    
will           : O    
take           : O    
reasonable     : O    
precautions    : O    
to             : O    
prevent        : O    
the            : O    
loss           : O    
of             : O    
or             : O    
alteration     : O    
to             : O    
Client         : O    
's             : O    
data           : O    
files          : O    
In             : O    
ADP            : O    
’              : O    
s              : O    
possassion     : O    
.              : O    
Client         : O    
wit            : O    
]              : O    
,              : O    
to             : O    
the            : O    
extent         : O    
it             : O    
deems          : O    
necessary  

7              : O    
.              : O    
Indemnification: O    
.              : O    
RIN            : O    
«              : O    
Container      : O    
Technolofies   : O    
7.1            : O    
.              : O    
Indemnification: O    
Agreement      : O    
.              : O    
Carrier        : O    
agree8         : O    
to             : O    
indemnify      : O    
,              : O    
defend         : O    
and            : O    
hold           : O    
Ring           : O    
and            : O    
its            : O    
Customer       : O    
(              : O    
s              : O    
)              : O    
(              : O    
including      : O    
their          : O    
officers       : O    
,              : O    
directors      : O    
,              : O    
and            : O    
employees      : O    
)              : O    
harmless       : O    
from           : O    
and            : O    
against        : O    
any            : O    
and        

1A2            : O    
.              : O    
Inspection     : O    
Cnntracter     : O    
agrees         : O    
that           : O    
,              : O    
at             : O    
all            : O    
times          : O    
during         : O    
the            : O    
Term           : O    
of             : O    
this           : O    
Agreement      : O    
,              : O    
Swifl          : O    
shall          : O    
have           : O    
the            : O    
right          : O    
to             : O    
make           : O    
inspections    : O    
of             : O    
the            : O    
Services       : O    
performed      : O    
by             : O    
Contractor     : O    
under          : O    
this           : O    
Agreement      : O    
.              : O    
Any            : O    
inspection     : O    
by             : O    
Swift          : O    
siiall         : O    
be             : O    
performed      : O    
so             : O    
as         

j.             : O    
Applicable     : O    
law            : O    
.              : O    
The            : O    
terms          : O    
of             : O    
any            : O    
license        : O    
agreement      : O    
entere         : O    
d              : O    
into           : O    
with           : O    
any            : O    
affiliate      : O    
of             : O    
ours           : O    
located        : O    
outside        : O    
of             : O    
Europe         : O    
will           : O    
be             : O    
governed       : O    
by             : O    
and            : O    
construed      : O    
in             : O    
accordance     : O    
with           : O    
the            : O    
laws           : O    
of             : O    
the            : O    
State          : O    
of             : O    
Washington     : I-gov
and            : O    
federal        : O    
laws           : O    
of             : O    
the            : O    
United     

...

ARTICLE        : O    
1              : O    
-              : O    
UNION          : O    
RECOGNITIONS   : O    
AND            : O    
CHECK          : O    
OFF            : O    
Section        : O    
1              : O    
-              : O    
The            : O    
Employer       : O    
recogpizes     : O    
Teamsters      : O    
Local          : O    
Union          : O    
NO             : O    
.              : O    
570            : O    
as             : O    
the            : O    
sole           : O    
and            : O    
exclusive      : O    
bargaining     : O    
agency         : O    
during         : O    
the            : O    
life           : O    
of             : O    
this           : O    
Agreement      : O    
for            : O    
its            : O    
employees      : O    
classified     : O    
herein         : O    
and            : O    
employed       : O    
in             : O    
its            : O    
Transportation : O    
Department 

1              : O    
.              : O    
Proiect        : O    
Services       : O    
.              : O    
UZURV          : O    
’              : O    
s              : O    
performance    : O    
of             : O    
Subcontracted  : O    
Services       : O    
for            : O    
specific       : O    
projects       : O    
shall          : O    
be             : O    
performed      : O    
in             : O    
accordance     : O    
with           : O    
the            : O    
terms          : O    
of             : O    
this           : O    
MSA            : O    
and            : O    
the            : O    
terms          : O    
of             : O    
separate       : O    
Task           : O    
Orders         : O    
to             : O    
be             : O    
executed       : O    
by             : O    
the            : O    
parties        : O    
.              : O    
In             : O    
the            : O    
event          : O    
of         

13.5           : O    
Governing      : O    
Law            : O    
;              : O    
Venue          : O    
.              : O    
This           : O    
Agreement      : O    
shall          : O    
be             : O    
construed      : O    
in             : O    
accordance     : O    
with           : O    
,              : O    
and            : O    
governed       : O    
in             : O    
all            : O    
respects       : O    
by             : O    
,              : O    
the            : B-gov
internal       : O    
laws           : I-gov
of             : I-gov
the            : I-gov
State          : I-gov
of             : I-gov
California     : I-gov
,              : I-gov
without        : O    
giving         : O    
effect         : O    
to             : O    
principles     : O    
of             : O    
conflicts      : O    
of             : O    
laws           : O    
provisions     : O    
thereof        : O    
.              : O    
['the', 'la

Article        : O    
24             : O    
-              : O    
Applicable     : O    
law            : O    
and            : O    
Jurisdiction   : O    
This           : O    
Agreement      : O    
is             : O    
governed       : O    
by             : O    
French         : B-gov
law            : I-gov
.              : I-gov
All            : O    
disputes       : O    
,              : O    
controversies  : O    
or             : O    
claims         : O    
arising        : O    
out            : O    
of             : O    
or             : O    
in             : O    
connection     : O    
with           : O    
this           : O    
Agreement      : O    
that           : O    
can            : O    
not            : O    
be             : O    
settled        : O    
amicably       : O    
between        : O    
the            : O    
Parties        : O    
shall          : O    
be             : O    
submitted      : O    
to             : O    
the        

10             : O    
,              : O    
Governing      : O    
Law            : O    
.              : O    
The            : O    
rights         : O    
end            : O    
obligations    : O    
of             : O    
the            : O    
parties        : O    
hereunder      : O    
shall          : O    
be             : O    
governed       : O    
by             : O    
and            : O    
construed      : O    
in             : O    
accordance     : O    
with           : O    
tie            : O    
laws           : O    
of             : O    
’              : O    
the            : O    
State          : O    
of             : O    
Delaware       : O    
,              : O    
without        : O    
i              : O    
’              : O    
cf             : O    
’              : O    
erence         : O    
to             : O    
its            : O    
choice         : O    
of             : O    
law            : O    
provisions     : O    
.          